# Vehicle Detection(training)

In [1]:
import numpy as np
import cv2
import glob
import random
import time
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn import decomposition

In [2]:
import vehicleDetectUtil as vehicleUtil
import vehicleDetect_svmVar as svmVar

In [3]:
def train():
    files_vehicle = glob.glob('vehicles/**/*.png', recursive=True)
    files_nonVehicle = glob.glob('non-vehicles/**/*.png', recursive=True)

    img = cv2.imread(files_vehicle[0])
    imgShape = img.shape

    #if svm:
    t=time.time()        
    files_vehicle_svm = files_vehicle
    files_nonVehicle_svm = files_nonVehicle
    print('SVM: using {} car and {} non-car images'.format(len(files_vehicle_svm), len(files_nonVehicle_svm)))
    print('SVM: preparing features...')
    car_features = vehicleUtil.extract_features(files_vehicle_svm, cspace=svmVar.spatial_clr, spatial_size=(svmVar.spatial, svmVar.spatial),
                            hist_bins=svmVar.histbin, hist_range=(0, 256), spatialFeat = svmVar.spatialFeat, histFeat = svmVar.histFeat,
                            hogFeat=svmVar.hogFeat, hog_cspace=svmVar.hog_clrspace, hog_orient=svmVar.orient, hog_pix_per_cell=svmVar.pix_per_cell, hog_cell_per_block=svmVar.cell_per_block, hog_channel=svmVar.hog_channel)
    notcar_features = vehicleUtil.extract_features(files_nonVehicle_svm, cspace=svmVar.spatial_clr, spatial_size=(svmVar.spatial, svmVar.spatial),
                            hist_bins=svmVar.histbin, hist_range=(0, 256), spatialFeat = svmVar.spatialFeat, histFeat = svmVar.histFeat,
                            hogFeat=svmVar.hogFeat, hog_cspace=svmVar.hog_clrspace, hog_orient=svmVar.orient, hog_pix_per_cell=svmVar.pix_per_cell, hog_cell_per_block=svmVar.cell_per_block, hog_channel=svmVar.hog_channel)
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    # Fit a per-column scaler
    print('SVM: normalizing features...')
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
        
    # singular value decomposition to reduce feature space
    pca = decomposition.PCA(n_components=3000)
    pca.fit(scaled_X)
    scaled_X = pca.transform(scaled_X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    t2=time.time()
    print('{} seconds to create {} feature vectors of size {}'.format(round(t2-t, 5), len(scaled_X), len(scaled_X[0])))
    print('SVM: splitting train/validation data...')
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.1, random_state=rand_state)

    print('SVM: Training model...')
    svc = LinearSVC()
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVM...')
    print('SVM: Saving model...')
    joblib.dump(svc, 'svm.pkl')
    joblib.dump(X_scaler, 'svm_scaler.pkl')
    joblib.dump(pca, 'svm_pca.pkl')
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

In [4]:
train()

SVM: using 8792 car and 8968 non-car images
SVM: preparing features...


/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


SVM: normalizing features...
193.10004 seconds to create 17760 feature vectors of size 3000
SVM: splitting train/validation data...
SVM: Training model...
6.63 Seconds to train SVM...
SVM: Saving model...
Test Accuracy of SVC =  0.9927
